In [ ]:
import sys
import cmath
import math
import os
import h5py
import matplotlib
import matplotlib.font_manager as fm
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt   # plots
import itertools
import numpy as np
import time
import warnings

from liblibra_core import *
import util.libutil as comn
from libra_py import units
import libra_py.models.Holstein as Holstein
import libra_py.models.Tully as Tully
import libra_py.models.Subotnik as Subotnik
import libra_py.models.Esch_Levine as Esch_Levine
from libra_py import dynamics_plotting

import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot
import libra_py.data_savers as data_savers

#from recipes import shxf, mqcxf, ehxf
#from recipes import ehrenfest_adi_ld, ehrenfest_adi_nac, ehrenfest_dia, mfsd
#from recipes import fssh, sdm, bcsh

import libra_py.dynamics.exact.compute as dvr
import libra_py.dynamics.exact.save as dvr_save

#from matplotlib.mlab import griddata
#%matplotlib inline 
warnings.filterwarnings('ignore')

## Color code
colors = {}
colors.update({"11": "#D20C0C", "12": "#FF5339", "13": "#FF8965"}) #red
colors.update({"21": "#00790C", "22": "#49A93F", "23": "#7DDB6D"}) #green
colors.update({"31": "#0009FF", "32": "#7145FF", "33": "#B176FF"}) #blue
colors.update({"41": "#FFA100", "42": "#FFD249", "43": "#FFFF7C"}) #orange
colors.update({"51": "#FF1DCE", "52": "#FFCDFF"}) #magenta
colors.update({"61": "#00ffea", "62": "#70FFFF", "63": "#AFFFFF"}) #cyan
colors.update({"71": "#bd20ff", "72": "#F461FF", "73": "#FF97FF"}) #indigo

clrs_index = ["11", "21", "31", "41", "51", "61", "71", "12", "22", "32", "42", "52", "62", "72" "13","23", "33", "53", "73"]


##  Total energy deviation, coherence, population: DAG/DC Holstein

In [ ]:
def compute_coherence(file):
    with h5py.File(file, "r") as f:
        C_ADI = f["Cadi/data"][:,:,:]

    nts, ntraj, nst = C_ADI.shape

    # Reshape C_ADI for better performance
    C_ADI_reshaped = C_ADI.reshape(nts, ntraj, nst, 1)

    # Compute rho2 using vectorized operations
    rho2 = np.sum(np.abs(C_ADI_reshaped) ** 2 * np.abs(C_ADI_reshaped.transpose(0, 1, 3, 2)) ** 2, axis=1) / ntraj

    return rho2

In [ ]:
dir_coh="data_coherence/"
if not os.path.exists(dir_coh):
    os.mkdir(dir_coh)

In [ ]:
# Compute and save coherence first
#list_methods = [6,7]
#model_indx = 5
#
#for i, _method_indx in enumerate(list_methods):
#    fn = F"model{model_indx}-method{_method_indx}-icond0/mem_data.hdf"
#    rho2 = compute_coherence(fn)
#
#    np.savetxt(dir_coh+F"coherence_model{model_indx}_method{_method_indx}_10.txt", rho2[:,1,0])

In [ ]:
def plot_ecp(_model_indx, list_methods = [6, 6, 7, 7], list_dirs = ["", "../", "", "../"],
             name_methods = ["SHXF, no BC", "SHXF, BC", "MQCXF, no BC", "MQCXF, BC"],
             title=None, markers = ["o", "D", "o", "D"], ms= [6, 6, 7, 7], zorder_methods = [11,10,11,10], list_ls=["-", "--", "dotted"],
             _nstates=2, _pad=-1, _bbox_to_anchor="", turn_on_legend=True):

    nst = _nstates

    fig, axs = plt.subplots(4, figsize=(10,12), gridspec_kw=dict(hspace=0), dpi=300)
    nr_mark = 30
    
    if title != None: axs[0].set_title(title, fontsize=28)
    
    # energy deviation
    Hart2eV=27.2114
    axs[0].set_ylabel(r"$|\Delta E|$ (mHa)", fontsize=28)
    axs[0].set_xticks([])
    axs[0].tick_params(axis='both', labelsize=28)
    #axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    axs[0].margins(x=0)
    
    # for legend
    axs[0].plot([0], [0], label='QD', color='k')
    axs[0].plot([], [], label=name_methods[0], color=colors[clrs_index[0]])
    axs[0].plot([], [], label=name_methods[1], color=colors[clrs_index[1]])
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
            axs[0].plot(f["time/data"][:]/41.0, np.abs(f["Etot_ave/data"][:]-f["Etot_ave/data"][0])*1000, linewidth=1, 
                label="", color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
    
    if turn_on_legend:
        if _bbox_to_anchor == "":
            axs[0].legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left').set_zorder(100)
        else:
            axs[0].legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left', bbox_to_anchor=_bbox_to_anchor).set_zorder(100)
    
    
    # Ehrenfest position
    
    # for legend
    axs[1].plot([], [], label=name_methods[3], color=colors[clrs_index[2]])
    axs[1].plot([], [], label=name_methods[3], color=colors[clrs_index[3]])
    axs[1].legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='center right').set_zorder(100)
    
    #axs[1].set_ylabel("position (a.u.)", fontsize=28)
    axs[1].set_ylabel(r"$\langle R \rangle$ (Bohr)", fontsize=28)
    #axs[1].set_yticks([-10,10,30])
    axs[1].set_xticks([])
    axs[1].tick_params(axis='both', labelsize=28)
    axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axs[1].margins(x=0)

    with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        axs[1].plot(f["time/data"][:]/41.0, f["q_adi/data"][:,0], linewidth=7, color="black", zorder=-10)
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
            axs[1].plot(f["time/data"][:]/41.0, np.average(f["q/data"][:,:,0], axis=1), 
            linewidth=1, label=name_methods[i], color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i]
            , zorder=zorder_methods[i])
    
    # coherences
    axs[2].set_ylabel("coherence", fontsize=28)
    axs[2].tick_params(axis='both', labelsize=28)
    axs[2].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axs[2].margins(x=0)
    axs[2].set_xticks([])
    with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        temp = "QD"
    
        cnt=-1
        for ist in range(nst):
            for jst in range(nst):
                if(ist<=jst): continue
                cnt+=1
                axs[2].plot(f["time/data"][:]/41.0, f["coherence_adi/data"][:, ist, jst], 
                            label=temp, linewidth=7, ls=list_ls[cnt], color = 'black', zorder=-10)
                temp = "_nolegend_"
        
        times = f["time/data"][:]
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
        
        #rho2 = compute_coherence(F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf")
        cnt=-1
        for ist in range(nst):
            for jst in range(nst):
                if(ist<=jst): continue
                cnt+=1
                rho2 = np.loadtxt(list_dirs[i] + dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_{ist}{jst}.txt")
                #axs[2].plot(times/41.0, rho2[:, ist, jst], linewidth=1, ls=list_ls[cnt],
                axs[2].plot(times/41.0, rho2, linewidth=1, ls=list_ls[cnt],
                    color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
    
    # populations
    axs[3].set_xlabel("time (fs)", fontsize=28)
    axs[3].set_ylabel("population", fontsize=28)
    axs[3].margins(x=0)
    axs[3].tick_params(axis='both', labelsize=28)
    #axs[3].set_yticks([0.0, 0.5, 1.0])
    
    with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        temp = "QD"
        #for ist in range(nst):
        #    axs[3].plot(f["time/data"][:]/41.0, f["pop_adi/data"][:, ist, 0], label=temp, linewidth=7, color = 'black', zorder=-10)
        axs[3].plot(f["time/data"][:]/41.0, f["pop_adi/data"][:, 1, 0], label=temp, linewidth=7, color = 'black', zorder=-10)
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
            
            #for ist in range(nst):
            #    axs[3].plot(f["time/data"][:]/41.0, f["se_pop_adi/data"][:, ist], linewidth=1, 
            #        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
            axs[3].plot(f["time/data"][:]/41.0, f["se_pop_adi/data"][:, 1], linewidth=1, 
                color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

    #fig.tight_layout(pad=_pad)
    fig.align_labels()
    fig.tight_layout()
    #axs[1].legend(fontsize=20)



In [ ]:
# Holstein
plot_ecp(5, title="DAG", _bbox_to_anchor=(-0.01,1.04), turn_on_legend=True)

In [ ]:
# DC Holstein
plot_ecp(3, title="DC Holstein", _bbox_to_anchor=(-0.01,1.04), turn_on_legend=False)

## Esch-Levine

In [ ]:
# Esch-Levine / Energy deviation, <R>
_model_indx = 7
list_methods = [6,6,7,7]
list_dirs = ["", "../", "", "../"]
name_methods = ["SHXF, no BC", "SHXF, BC", "MQCXF, no BC", "MQCXF, BC"]
markers = ["o", "D", "o", "D"]
ms = [6, 7, 6, 7]
zorder_methods = [11,10,11,10]
_tlim=195.1
_slim=8000

nst = 3
nr_mark = 30

#fig, axs = plt.subplots(5, figsize=(10,10), gridspec_kw={'hspace': 0, 'height_ratios': [1,1,1,1,2.5]})
fig, axs = plt.subplots(2, figsize=(11,10), gridspec_kw={'hspace': 0}, dpi=300)
# energy deviation
Hart2eV=27.2114
axs[0].set_ylabel(r'$|\Delta E|$ (mHa)', fontsize=28)
#axs[0].set_ylim([0, 3.5])
axs[0].set_xlim([0, _tlim])
axs[0].set_xticks([])
axs[0].tick_params(axis='both', labelsize=28)
axs[0].margins(x=0)

axs[0].plot([0], [0], label='QD', color='k')
for i, _method_indx in enumerate(list_methods):
    fn = list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[0].plot(f["time/data"][:_slim]/41.0, np.abs(f["Etot_ave/data"][:_slim]-f["Etot_ave/data"][0])*1000, linewidth=1,
            label=name_methods[i], color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
        
axs[0].legend(fontsize=28, ncol=2, columnspacing=0.5, loc='upper left', frameon=False).set_zorder(100)

# position
#axs[1].set_ylabel("position (a.u.)", fontsize=25)
axs[1].set_ylabel(r"$\langle R \rangle$ (Bohr)", fontsize=28)
axs[1].set_ylim([-5, 39])
axs[1].set_xlim([0, _tlim])
#axs[1].set_xticks([])
axs[1].tick_params(axis='both', labelsize=28)
axs[1].set_xlabel("time (fs)", fontsize=28)
#axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
axs[1].margins(x=0)

with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    axs[1].plot(f["time/data"][:_slim]/41.0, f["q_adi/data"][:_slim,0], linewidth=7, color="black", zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[1].plot(f["time/data"][:_slim]/41.0, np.average(f["q/data"][:_slim,:,0], axis=1), 
        linewidth=1, color = colors[clrs_index[i]],  marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

fn = list_dirs[i] + F"model7-method7-icond0/mem_data.hdf"
with h5py.File(fn, 'r') as f:
    axs[1].plot(f["time/data"][:]/41.0, np.average(f["q/data"][:,:,0], axis=1),
    linewidth=2, linestyle="dashed", color = colors["31"], zorder=5)
        

fig.align_labels()
fig.tight_layout()

In [ ]:
# Esch-Levine
_model_indx = 7
list_methods = [6,6,7,7]
list_dirs = ["", "../", "", "../"]
name_methods = ["SHXF, no BC", "SHXF, BC", "MQCXF, no BC", "MQCXF, BC"]
markers = ["o", "D", "o", "D"]
ms = [6, 7, 6, 7]
zorder_methods = [11,10,11,10]
_tlim=195.1
_slim=8000

nst = 3
nr_mark = 30

#fig, axs = plt.subplots(5, figsize=(10,10), gridspec_kw={'hspace': 0, 'height_ratios': [1,1,1,1,2.5]})
fig, axs = plt.subplots(3, figsize=(11,10), gridspec_kw={'hspace': 0}, dpi=300)

# populations, 2
axs[0].set_ylabel(r"$\langle \rho_{2}\rangle$", fontsize=28)
axs[0].tick_params(axis='both', labelsize=28)
axs[0].set_xlim([0, _tlim])
axs[0].margins(x=0)
axs[0].set_xticks([])
axs[0].set_ylim([0.3, 1.06])
axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

# for legend
axs[0].plot([],[], color="k", label="QD")
axs[0].plot([],[], color=colors[clrs_index[0]], label=name_methods[0], marker=markers[0], markersize=ms[0])
axs[0].plot([],[], color=colors[clrs_index[1]], label=name_methods[1], marker=markers[0], markersize=ms[0])

with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[0].plot(f["time/data"][:_slim]/41.0, f["pop_adi/data"][:_slim, 2, 0], label="", linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
        
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[0].plot(f["time/data"][:_slim]/41.0, f["se_pop_adi/data"][:_slim, 2], linewidth=1, label="",
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

axs[0].legend(fontsize=26, ncol=3, columnspacing=0.5, loc='upper right', frameon=False).set_zorder(100)
        
# populations, 1

# for legend
axs[1].plot([],[], color=colors[clrs_index[2]], label=name_methods[2], marker=markers[0], markersize=ms[0])
axs[1].plot([],[], color=colors[clrs_index[3]], label=name_methods[3], marker=markers[0], markersize=ms[0])
axs[1].legend(fontsize=26, ncol=3, columnspacing=0.5, loc='upper right', frameon=False).set_zorder(100)

axs[1].set_ylabel(r"$\langle \rho_{1}\rangle$", fontsize=28)
axs[1].tick_params(axis='both', labelsize=28)
axs[1].set_xlim([0, _tlim])
axs[1].margins(x=0)
axs[1].set_xticks([])
axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[1].plot(f["time/data"][:_slim]/41.0, f["pop_adi/data"][:_slim, 1, 0], label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
        
    with h5py.File(fn, 'r') as f:
        _nstep =_slim
        freq = int(_nstep/nr_mark)
        axs[1].plot(f["time/data"][:_slim]/41.0, f["se_pop_adi/data"][:_slim, 1], linewidth=1, 
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# populations, 0
axs[2].set_xlabel("time (fs)", fontsize=28)
axs[2].set_ylabel(r"$\langle \rho_{0}\rangle$", fontsize=28)
axs[2].tick_params(axis='both', labelsize=28)
axs[2].set_xticks([0, 50, 100, 150])
axs[2].set_xlim([0, _tlim])
axs[2].margins(x=0)
axs[2].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[2].plot(f["time/data"][:_slim]/41.0, f["pop_adi/data"][:_slim, 0, 0], label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
        
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        axs[2].plot(f["time/data"][:_slim]/41.0, f["se_pop_adi/data"][:_slim, 0], linewidth=1, 
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
#fig.tight_layout(pad=_pad)
fig.align_labels()
fig.tight_layout()

In [ ]:
# Esch-Levine
_model_indx = 7
list_methods = [6,6,7,7]
list_dirs = ["", "../", "", "../"]
name_methods = ["SHXF, no BC", "SHXF, BC", "MQCXF, no BC", "MQCXF, BC"]
markers = ["o", "D", "o", "D"]
ms = [6, 7, 6, 7]
zorder_methods = [11,10,11,10]
_tlim=195.1
_slim=8000

nst = 3
nr_mark = 30

#fig, axs = plt.subplots(5, figsize=(10,10), gridspec_kw={'hspace': 0, 'height_ratios': [1,1,1,1,2.5]})
fig, axs = plt.subplots(3, figsize=(11,10), gridspec_kw={'hspace': 0}, dpi=300)
        
# coherences, 12
axs[0].set_ylabel(r"$\langle |\rho_{12}|^{2} \rangle $", fontsize=28)
axs[0].tick_params(axis='both', labelsize=28)
axs[0].set_xlim([0, _tlim])
axs[0].margins(x=0)
axs[0].set_xticks([])
axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[0].plot(f["time/data"][:_slim]/41.0, f["coherence_adi/data"][:_slim, 2, 1], label=temp, linewidth=7, color = 'black', zorder=-10)
    
for i, _method_indx in enumerate(list_methods):
    fn = list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
        times = f["time/data"][:_slim]
    #rho2 = compute_coherence(fn)
    rho2 = np.loadtxt(list_dirs[i] + dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_21.txt")
    #print(times.shape)
    #print(rho2.shape)
    axs[0].plot(times/41.0, rho2[:_slim], linewidth=1,
            color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# coherences, 01
axs[1].set_ylabel(r"$\langle |\rho_{01}|^{2} \rangle $", fontsize=28)
axs[1].tick_params(axis='both', labelsize=28)
axs[1].set_xlim([0, _tlim])
axs[1].margins(x=0)
axs[1].set_xticks([])
axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[1].plot(f["time/data"][:_slim]/41.0, f["coherence_adi/data"][:_slim, 1, 0], 
        label=temp, linewidth=7, color = 'black', zorder=-10)
    
for i, _method_indx in enumerate(list_methods):
    fn = list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
    #rho2 = compute_coherence(fn)
    rho2 = np.loadtxt(list_dirs[i] + dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_10.txt")
    axs[1].plot(times/41.0, rho2[:_slim], linewidth=1,
        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

# coherences, 02
axs[2].set_ylabel(r"$\langle |\rho_{02}|^{2} \rangle $", fontsize=28)
axs[2].tick_params(axis='both', labelsize=28)
axs[2].set_xlim([0, _tlim])
axs[2].margins(x=0)
#axs[2].set_xticks([])
axs[2].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    temp = "QD"
    axs[2].plot(f["time/data"][:_slim]/41.0, f["coherence_adi/data"][:_slim, 2, 0], 
        label=temp, linewidth=7, color = 'black', zorder=-10)

for i, _method_indx in enumerate(list_methods):
    fn = list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf"
    with h5py.File(fn, 'r') as f:
        _nstep = _slim
        freq = int(_nstep/nr_mark)
    #rho2 = compute_coherence(fn)
    rho2 = np.loadtxt(list_dirs[i] + dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_20.txt")
    axs[2].plot(times/41.0, rho2[:_slim], linewidth=1,
        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
axs[2].set_xlabel("time (fs)", fontsize=28)

fig.align_labels()
fig.tight_layout()

## Energy deviation

In [ ]:
list_nsteps = [8000, 8000, 4000, 3000, 8000]

In [ ]:
def plot_dE(list_methods = [7, 7], list_dirs = ["", "../"],
             name_methods = ["no BC", "BC"], list_models = [1, 3, 4, 5, 7],
             name_models = ["SC Holsten", "DC Holstein", "ECWR", "DAG", "Esch-Levine"],
             title=None, markers = ["o", "D"], ms= [8, 10], zorder_methods = [11,10], list_ls=["-", "--", "dotted"],
             _pad=-1, _bbox_to_anchor="", turn_on_legend=True):

    fig, axs = plt.subplots(figsize=(10,8), dpi=300)
    nr_mark = 10
    
    if title != None: axs.set_title(title, fontsize=28)
    
    # energy deviation
    Hart2eV=27.2114
    axs.set_xlabel("time (fs)", fontsize=28)
    axs.set_ylabel(r"$|\Delta E/E(0)|$", fontsize=28)
    #axs.set_yticks([-1, -2, -5, -10, -15])
    axs.tick_params(axis='both', labelsize=28)
    #axs.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    axs.margins(x=0)
    
    # for label
    for i, _model_indx in enumerate(list_methods):
        axs.plot([], [], label = F"{name_methods[i]}", color = "k", marker=markers[i], markersize=ms[i], ls=list_ls[i], lw=3)
    
    for i, _model_indx in enumerate(list_models):
        axs.plot([], [], label = F"{name_models[i]}", color = colors[clrs_index[i]], lw=3)

    
    # no BC
    for i, _model_indx in enumerate(list_models):
        with h5py.File(list_dirs[0] + F"model{_model_indx}-method{list_methods[0]}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:list_nsteps[i]])
            freq = int(8000/nr_mark)
            dE_E = np.abs((f["Etot_ave/data"][:list_nsteps[i]]-f["Etot_ave/data"][0])/f["Etot_ave/data"][0])
            axs.plot(f["time/data"][:list_nsteps[i]]/41.0, dE_E[:list_nsteps[i]], linewidth=3, 
                color = colors[clrs_index[i]], ls=list_ls[0], 
                marker=markers[0], markevery=[x for x in range(0, _nstep, freq)] + [_nstep-1], markersize=ms[0], zorder=zorder_methods[0])
    # BC
    for i, _model_indx in enumerate(list_models):
        with h5py.File(list_dirs[1] + F"model{_model_indx}-method{list_methods[1]}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:list_nsteps[i]])
            freq = int(8000/nr_mark)
            dE_E = np.abs((f["Etot_ave/data"][:list_nsteps[i]]-f["Etot_ave/data"][0])/f["Etot_ave/data"][0])
            axs.plot(f["time/data"][:list_nsteps[i]]/41.0, dE_E[:list_nsteps[i]], linewidth=3, 
                color = colors[clrs_index[i]], ls=list_ls[1], 
                marker=markers[1], markevery=[x for x in range(0, _nstep, freq)] + [_nstep-1], markersize=ms[1], zorder=zorder_methods[1])
            
    if turn_on_legend:
        if _bbox_to_anchor == "":
            axs.legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left').set_zorder(100)
        else:
            axs.legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left', bbox_to_anchor=_bbox_to_anchor).set_zorder(100)
    
    
    #fig.tight_layout(pad=_pad)
    fig.tight_layout()
    fig.align_labels()
    fig.tight_layout()
    #axs[1].legend(fontsize=20)



In [ ]:
plot_dE(_bbox_to_anchor=(-0.01,1.03))

In [ ]:
plot_dE(list_methods =[6,6], _bbox_to_anchor=(-0.01,1.03))

In [ ]:
def plot_dR(list_methods = [7, 7], list_dirs = ["", "../"],
             name_methods = ["no BC", "BC"], list_models = [1, 3, 4, 5, 7],
             name_models = ["SC Holsten", "DC Holstein", "ECWR", "DAG", "Esch-Levine"],
             title=None, markers = ["o", "D"], ms= [8, 10], zorder_methods = [11,10], list_ls=["-", "--", "dotted"],
             _pad=-1, _bbox_to_anchor="", turn_on_legend=True):

    fig, axs = plt.subplots(figsize=(10,8), dpi=300)
    nr_mark = 10
    
    if title != None: axs.set_title(title, fontsize=28)
    
    # R
    axs.set_xlabel("time (fs)", fontsize=28)
    axs.set_ylabel(r"$|\Delta \langle R \rangle |$ (Bohr)", fontsize=28)
    #axs.set_yticks([1, 0, -1, -2, -5, -10, -15])
    axs.tick_params(axis='both', labelsize=28)
    #axs.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    axs.margins(x=0)
    
    # for label
    for i, _model_indx in enumerate(list_methods):
        axs.plot([], [], label = F"{name_methods[i]}", color = "k", marker=markers[i], markersize=ms[i])
    
    for i, _model_indx in enumerate(list_models):
        axs.plot([], [], label = F"{name_models[i]}", color = colors[clrs_index[i]])

    
    # no BC
    for i, _model_indx in enumerate(list_models):
        prf = F"../exact-model{_model_indx}-icond0"
    
        with h5py.File(F"{prf}/data.hdf", 'r') as F:
            q_adi_exact = np.array(F["q_adi/data"][:list_nsteps[i], 0, 0])              
            t = np.array(F['time/data'][:list_nsteps[i]])
        
        with h5py.File(list_dirs[0] + F"model{_model_indx}-method{list_methods[0]}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:list_nsteps[i]])
            freq = int(8000/nr_mark)
            x = np.array(f["q/data"][:list_nsteps[i],:,0])
            x = np.average(x, axis=1)
            
            dx = np.abs((x[:] - q_adi_exact[:]) )
            axs.plot(f["time/data"][:list_nsteps[i]]/41.0, dx[:list_nsteps[i]], linewidth=3, 
                color = colors[clrs_index[i]], ls=list_ls[0], 
                marker=markers[0], markevery=[x for x in range(0, _nstep, freq)] + [_nstep-1], markersize=ms[0], zorder=zorder_methods[0])

    # BC
    for i, _model_indx in enumerate(list_models):
        prf = F"../exact-model{_model_indx}-icond0"
    
        with h5py.File(F"{prf}/data.hdf", 'r') as F:
            q_adi_exact = np.array(F["q_adi/data"][:list_nsteps[i], 0, 0])              
            t = np.array(F['time/data'][:list_nsteps[i]])
        
        with h5py.File(list_dirs[1] + F"model{_model_indx}-method{list_methods[1]}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:list_nsteps[i]])
            freq = int(8000/nr_mark)
            x = np.array(f["q/data"][:list_nsteps[i],:,0])
            x = np.average(x, axis=1)
            
            dx = np.abs((x[:] - q_adi_exact[:]) )
            axs.plot(f["time/data"][:list_nsteps[i]]/41.0, dx[:list_nsteps[i]], linewidth=3, 
                color = colors[clrs_index[i]], ls=list_ls[1], 
                marker=markers[1], markevery=[x for x in range(0, _nstep, freq)] + [_nstep-1], markersize=ms[1], zorder=zorder_methods[1])

    if turn_on_legend:
        if _bbox_to_anchor == "":
            axs.legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left').set_zorder(100)
        else:
            axs.legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left', bbox_to_anchor=_bbox_to_anchor).set_zorder(100)
    
    
    #fig.tight_layout(pad=_pad)
    #fig.align_labels()
    #fig.tight_layout()
    #axs[1].legend(fontsize=20)



In [ ]:
plot_dR(title="MQCXF", turn_on_legend=False)

In [ ]:
plot_dR(title="SHXF", list_methods = [6,6])

In [ ]:
def plot_dR(list_methods = [], list_dirs = ["", "../"],
             name_methods = ["no BC", "BC"], list_models = [1, 3, 4, 5, 7],
             name_models = ["SC Holsten", "DC Holstein", "ECWR", "DAG", "Esch-Levine"],
             title=None, markers = ["o", "D"], ms= [8, 10], zorder_methods = [11,10], list_ls=["-", "--", "dotted"],
             _pad=-1, _bbox_to_anchor="", turn_on_legend=True):

    fig, axs = plt.subplots(2, figsize=(10,8), gridspec_kw={'hspace': 0}, dpi=300)
    nr_mark = 10
    
    list_methods = [6, 6]
    if title != None: axs.set_title(title, fontsize=28)
    
    # R
    axs[0].set_xlabel("time (fs)", fontsize=28)
    axs[0].set_ylabel(r"$|\Delta \langle R \rangle |$ (Bohr)", fontsize=28)
    axs[0].set_xticks([])
    axs[0].set_ylim([-0.2, 6.7])
    #axs.set_yticks([1, 0, -1, -2, -5, -10, -15])
    axs[0].tick_params(axis='both', labelsize=28)
    #axs.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    axs[0].margins(x=0)
    
    # for label
    #for i, _method_indx in enumerate(list_methods):
    #    axs[0].plot([], [], label = F"{name_methods[i]}", color = "k", marker=markers[i], markersize=ms[i])
    axs[0].text(165, 5.5, "SHXF", fontsize=28)
    
    # no BC
    for i, _model_indx in enumerate(list_models):
        prf = F"../exact-model{_model_indx}-icond0"
    
        with h5py.File(F"{prf}/data.hdf", 'r') as F:
            q_adi_exact = np.array(F["q_adi/data"][:list_nsteps[i], 0, 0])              
            t = np.array(F['time/data'][:list_nsteps[i]])
        
        with h5py.File(list_dirs[0] + F"model{_model_indx}-method{list_methods[0]}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:list_nsteps[i]])
            freq = int(8000/nr_mark)
            x = np.array(f["q/data"][:list_nsteps[i],:,0])
            x = np.average(x, axis=1)
            
            dx = np.abs((x[:] - q_adi_exact[:]) )
            axs[0].plot(f["time/data"][:list_nsteps[i]]/41.0, dx[:list_nsteps[i]], linewidth=3, 
                color = colors[clrs_index[i]], ls=list_ls[0], 
                marker=markers[0], markevery=[x for x in range(0, _nstep, freq)] + [_nstep-1], markersize=ms[0], zorder=zorder_methods[0])

    # BC
    for i, _model_indx in enumerate(list_models):
        prf = F"../exact-model{_model_indx}-icond0"
    
        with h5py.File(F"{prf}/data.hdf", 'r') as F:
            q_adi_exact = np.array(F["q_adi/data"][:list_nsteps[i], 0, 0])              
            t = np.array(F['time/data'][:list_nsteps[i]])
        
        with h5py.File(list_dirs[1] + F"model{_model_indx}-method{list_methods[1]}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:list_nsteps[i]])
            freq = int(8000/nr_mark)
            x = np.array(f["q/data"][:list_nsteps[i],:,0])
            x = np.average(x, axis=1)
            
            dx = np.abs((x[:] - q_adi_exact[:]) )
            axs[0].plot(f["time/data"][:list_nsteps[i]]/41.0, dx[:list_nsteps[i]], linewidth=3, 
                color = colors[clrs_index[i]], ls=list_ls[1], 
                marker=markers[1], markevery=[x for x in range(0, _nstep, freq)] + [_nstep-1], markersize=ms[1], zorder=zorder_methods[1])

    #if turn_on_legend:
    #    if _bbox_to_anchor == "":
    #        axs[0].legend(fontsize=28, ncol=2, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left').set_zorder(100)
    #    else:
    #        axs[0].legend(fontsize=28, ncol=2, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left', bbox_to_anchor=_bbox_to_anchor).set_zorder(100)

    ##### MQCXF
    
    list_methods = [7, 7]
    if title != None: axs.set_title(title, fontsize=28)
    
    # R
    axs[1].set_xlabel("time (fs)", fontsize=28)
    axs[1].set_ylabel(r"$|\Delta \langle R \rangle |$ (Bohr)", fontsize=28)
    axs[1].set_ylim([-0.2, 6.7])
    #axs.set_yticks([1, 0, -1, -2, -5, -10, -15])
    axs[1].tick_params(axis='both', labelsize=28)
    #axs.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    axs[1].margins(x=0)
    
    # for label
    #for i, _model_indx in enumerate(list_models):
    #    axs[1].plot([], [], label = F"{name_models[i]}", color = colors[clrs_index[i]])
    axs[1].text(160, 5.5, "MQCXF", fontsize=28)
    
    # no BC
    for i, _model_indx in enumerate(list_models):
        prf = F"../exact-model{_model_indx}-icond0"
    
        with h5py.File(F"{prf}/data.hdf", 'r') as F:
            q_adi_exact = np.array(F["q_adi/data"][:list_nsteps[i], 0, 0])              
            t = np.array(F['time/data'][:list_nsteps[i]])
        
        with h5py.File(list_dirs[0] + F"model{_model_indx}-method{list_methods[0]}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:list_nsteps[i]])
            freq = int(8000/nr_mark)
            x = np.array(f["q/data"][:list_nsteps[i],:,0])
            x = np.average(x, axis=1)
            
            dx = np.abs((x[:] - q_adi_exact[:]) )
            axs[1].plot(f["time/data"][:list_nsteps[i]]/41.0, dx[:list_nsteps[i]], linewidth=3, 
                color = colors[clrs_index[i]], ls=list_ls[0], 
                marker=markers[0], markevery=[x for x in range(0, _nstep, freq)] + [_nstep-1], markersize=ms[0], zorder=zorder_methods[0])

    # BC
    for i, _model_indx in enumerate(list_models):
        prf = F"../exact-model{_model_indx}-icond0"
    
        with h5py.File(F"{prf}/data.hdf", 'r') as F:
            q_adi_exact = np.array(F["q_adi/data"][:list_nsteps[i], 0, 0])              
            t = np.array(F['time/data'][:list_nsteps[i]])
        
        with h5py.File(list_dirs[1] + F"model{_model_indx}-method{list_methods[1]}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:list_nsteps[i]])
            freq = int(8000/nr_mark)
            x = np.array(f["q/data"][:list_nsteps[i],:,0])
            x = np.average(x, axis=1)
            
            dx = np.abs((x[:] - q_adi_exact[:]) )
            axs[1].plot(f["time/data"][:list_nsteps[i]]/41.0, dx[:list_nsteps[i]], linewidth=3, 
                color = colors[clrs_index[i]], ls=list_ls[1], 
                marker=markers[1], markevery=[x for x in range(0, _nstep, freq)] + [_nstep-1], markersize=ms[1], zorder=zorder_methods[1])

    #if turn_on_legend:
    #    if _bbox_to_anchor == "":
    #        axs[1].legend(fontsize=28, ncol=2, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left').set_zorder(100)
    #    else:
    #        axs[1].legend(fontsize=28, ncol=2, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper left', bbox_to_anchor=_bbox_to_anchor).set_zorder(100)
    
    fig.tight_layout(pad=_pad)
    fig.align_labels()
    fig.tight_layout()
    #axs[1].legend(fontsize=20)



In [ ]:
plot_dR()